# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
! pip install requests

In [2]:
! pip install gmaps

In [3]:
! jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import (weather_api_key, g_key)

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
# Load the csv
cities_vacation = pd.read_csv("Resources/cities_vacation.csv")
cities_vacation

,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date
0,Hobart,-42.88,147.33,60.80,51,16.11,0,AU,1601635838
1,Lavrentiya,65.58,-171.00,40.46,94,10.16,100,RU,1601636119
2,Rikitea,-23.12,-134.97,72.54,81,19.13,0,PF,1601636121
3,Ullapool,57.90,-5.16,54.00,76,11.99,92,GB,1601636121
4,Hithadhoo,-0.60,73.08,84.24,65,5.35,80,MV,1601636122
...,...,...,...,...,...,...,...,...,...
509,Olafsvik,64.89,-23.71,44.11,79,20.51,100,IS,1601636553
510,Jacareacanga,-6.22,-57.75,74.07,91,2.30,97,BR,1601636554
511,Hvide Sande,56.00,8.13,63.00,93,12.75,75,DK,1601636555
512,Husavik,66.04,-17.34,24.80,85,3.36,100,IS,1601636555


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Store latitude and longitude in locations
locations = cities_vacation[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = cities_vacation["Humidity"].astype(float)

In [7]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Drop the cities rows where the temperature is higher than 80
cities_vacation_1 = cities_vacation[cities_vacation['Max Temp'] > 78].index

# Delete these row indexes from dataFrame
cities_vacation.drop(cities_vacation_1 , inplace=True)

cities_vacation

,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date
0,Hobart,-42.88,147.33,60.80,51,16.11,0,AU,1601635838
1,Lavrentiya,65.58,-171.00,40.46,94,10.16,100,RU,1601636119
2,Rikitea,-23.12,-134.97,72.54,81,19.13,0,PF,1601636121
3,Ullapool,57.90,-5.16,54.00,76,11.99,92,GB,1601636121
6,Tuktoyaktuk,69.45,-133.04,36.88,95,4.56,91,CA,1601636123
...,...,...,...,...,...,...,...,...,...
509,Olafsvik,64.89,-23.71,44.11,79,20.51,100,IS,1601636553
510,Jacareacanga,-6.22,-57.75,74.07,91,2.30,97,BR,1601636554
511,Hvide Sande,56.00,8.13,63.00,93,12.75,75,DK,1601636555
512,Husavik,66.04,-17.34,24.80,85,3.36,100,IS,1601636555


In [9]:
# Drop the cities rows where the temperature is lower than 70
cities_vacation_2 = cities_vacation[cities_vacation['Max Temp'] < 70].index

# Delete these row indexes from dataFrame
cities_vacation.drop(cities_vacation_2 , inplace=True)

cities_vacation

,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date
2,Rikitea,-23.12,-134.97,72.54,81,19.13,0,PF,1601636121
9,Fort-Shevchenko,44.51,50.26,70.88,29,11.86,2,KZ,1601636125
23,Atambua,-9.11,124.89,75.45,78,3.62,43,ID,1601636137
28,Saint-Philippe,-21.36,55.77,77.00,73,9.17,90,RE,1601636141
29,Imbituba,-28.24,-48.67,75.20,78,14.99,1,BR,1601636142
...,...,...,...,...,...,...,...,...,...
471,Poum,-20.23,164.02,71.58,72,23.96,45,NC,1601636522
493,San Jose,37.34,-121.89,73.99,52,3.36,1,US,1601636401
507,Santa Maria,-29.68,-53.81,70.74,90,12.19,100,BR,1601636330
508,La Palma,33.85,-118.05,71.60,73,1.83,1,US,1601636553


In [10]:
# Drop the cities rows where the wind speed is over 10 mph
cities_vacation_3 = cities_vacation[cities_vacation['Wind Speed'] > 10].index

# Delete these row indexes from dataFrame
cities_vacation.drop(cities_vacation_3 , inplace=True)

cities_vacation

,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date
23,Atambua,-9.11,124.89,75.45,78,3.62,43,ID,1601636137
28,Saint-Philippe,-21.36,55.77,77.00,73,9.17,90,RE,1601636141
36,Muroto-misakicho,33.28,134.15,73.40,73,3.36,75,JP,1601636147
41,Bengkulu,-3.80,102.27,77.72,83,2.04,55,ID,1601636151
58,Wukari,7.85,9.78,76.42,87,3.40,98,NG,1601636167
67,Inhambane,-23.86,35.38,75.20,94,2.24,75,MZ,1601636175
68,Alofi,-19.06,-169.92,75.20,94,3.36,64,NU,1601636176
73,Grand Gaube,-20.01,57.66,77.00,68,5.01,1,MU,1601636180
75,San Ignacio,17.16,-89.07,73.00,89,2.42,96,BZ,1601636181
99,Duekoué,6.75,-7.35,75.79,88,5.06,100,CI,1601636200


In [11]:
# Drop the cities rows where the cloudiness is greater than 5
cities_vacation_4 = cities_vacation[cities_vacation['Cloudiness'] > 0].index

# Delete these row indexes from dataFrame
cities_vacation.drop(cities_vacation_4, inplace=True)

cities_vacation

,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date
100,San Patricio,19.22,-104.70,76.24,80,4.43,0,MX,1601636201
105,Guerrero Negro,27.98,-114.06,73.31,64,4.65,0,MX,1601636205
142,Hervey Bay,-25.30,152.85,70.00,75,1.01,0,AU,1601636235
183,Baharly,38.44,57.43,76.73,17,7.34,0,TM,1601636267
291,Konya Province,38.17,32.50,71.60,33,4.70,0,TR,1601636350
450,Ures,29.43,-110.40,77.00,81,1.99,0,MX,1601636502
462,Afşin,38.25,36.91,76.15,19,6.46,0,TR,1601636513
467,Camiri,-20.05,-63.52,71.76,50,0.69,0,BO,1601636517


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
# set up additional columns to hold information
cities_vacation['Hotel Name'] = ""

cities_vacation

,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date,Hotel Name
100,San Patricio,19.22,-104.70,76.24,80,4.43,0,MX,1601636201,
105,Guerrero Negro,27.98,-114.06,73.31,64,4.65,0,MX,1601636205,
142,Hervey Bay,-25.30,152.85,70.00,75,1.01,0,AU,1601636235,
183,Baharly,38.44,57.43,76.73,17,7.34,0,TM,1601636267,
291,Konya Province,38.17,32.50,71.60,33,4.70,0,TR,1601636350,
450,Ures,29.43,-110.40,77.00,81,1.99,0,MX,1601636502,
462,Afşin,38.25,36.91,76.15,19,6.46,0,TR,1601636513,
467,Camiri,-20.05,-63.52,71.76,50,0.69,0,BO,1601636517,


In [24]:
# Use the lat/lng to identify cities
for index, row in cities_vacation.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Set the condition and find the first hotel of each type to coordinates
    params = {
        "radius": 5000,
        "types": "hotel",
        "keyword": "hotel",
        "key": g_key
    }

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url and convert to JSON
    name_address = requests.get(base_url, params=params).json()

    # Save the first hotel names into the column
    try:
        cities_vacation.loc[index, "Hotel Name"] = name_address["results"][0]["name"]

    # If no hotel around, then show the message below.
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.


In [26]:
cities_vacation

,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date,Hotel Name
100,San Patricio,19.22,-104.70,76.24,80,4.43,0,MX,1601636201,Grand Isla Navidad Resort
105,Guerrero Negro,27.98,-114.06,73.31,64,4.65,0,MX,1601636205,Hotel Los Caracoles
142,Hervey Bay,-25.30,152.85,70.00,75,1.01,0,AU,1601636235,Oaks Hervey Bay Resort and Spa
183,Baharly,38.44,57.43,76.73,17,7.34,0,TM,1601636267,Baharly
291,Konya Province,38.17,32.50,71.60,33,4.70,0,TR,1601636350,Bahçesaray
450,Ures,29.43,-110.40,77.00,81,1.99,0,MX,1601636502,Hacienda del Labrador
462,Afşin,38.25,36.91,76.15,19,6.46,0,TR,1601636513,Meka Royal Otel
467,Camiri,-20.05,-63.52,71.76,50,0.69,0,BO,1601636517,Hotel Gran Capital


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in cities_vacation.iterrows()]
locations = cities_vacation[["Lat", "Lng"]]

In [19]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))